In [119]:
!pip install watson-machine-learning-client-V4

In [120]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [121]:
wml_credentials = {
    "apikey": "SLX4l5rDsGs8Y2gCWFeDNyC16Hk5DPWmOoITnMr6Lj-8",
    "instance_id": "427ba6b1-1adc-464f-be84-4203fd4321f7",
    "url": "https://us-south.ml.cloud.ibm.com",
}

In [122]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [123]:
log('Authenticating for WML')

def loginWML():
    data = 'apikey='+wml_credentials["apikey"]+'&grant_type=urn%3Aibm%3Aparams%3Aoauth%3Agrant-type%3Aapikey'
    myauthheaders = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': 'Basic Yng6Yng='
    }
    r = requests.post("https://iam.cloud.ibm.com/identity/token", headers=myauthheaders, data=data)
    res = json.loads(r.text)
    return res['access_token']
    
wml_token = loginWML();

log('Getting DO runtime')

mywmlheaders = {
    'Authorization': 'Bearer '+wml_token,
    'ML-Instance-ID': wml_credentials["instance_id"],
    'Content-Type': 'application/json',
    'cache-control': 'no-cache'
}

def getDORuntime():    
    r = requests.get(wml_credentials["url"] + '/v4/runtimes?runtime.name=do_12.9', headers=mywmlheaders)
    res = json.loads(r.text)
    return res['resources'][0]['metadata']['href']

#runtime = getDORuntime()
runtime = '/v4/runtimes/do_12.9'

print ("runtime: "+runtime)

Authenticating for WML 		Duration: 71.38 	Total: 5086.21 (sec.)
Getting DO runtime 		Duration: 0.4 	Total: 5086.61 (sec.)
runtime: /v4/runtimes/do_12.9


In [124]:
cos_credentials = {
    "apikey": "j-K5n-vX4joK8TDmCOcwxebCcGOxEC8G_U1JuOg2-JM6",
    "bucket": "productionoplmodel-donotdelete-pr-pi3c8bpsesbd17",
    "key": "a544085e34dff72513edfdcfc0052800b8bce17f3b5100d6",
    "id": "b423b2554da648a4b9204ebf0f6e3074",
    "url": "https://s3-api.us-geo.objectstorage.softlayer.net",
}

In [125]:
log('Authenticating for Cloud Object Storage')

def loginCOS():

    myiamheaders = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    payload = {
        'apikey': cos_credentials["apikey"],
        'response_type': 'cloud_iam',
        'grant_type':'urn:ibm:params:oauth:grant-type:apikey'
    }

    r = requests.post('https://iam.cloud.ibm.com/oidc/token', headers=myiamheaders, data=payload)

    res = json.loads(r.text)
    return res['access_token']

cos_token = loginCOS();

mycosheaders = {
    'Authorization': 'bearer ' + cos_token,
    'Content-Type': 'text/plain'
}

Authenticating for Cloud Object Storage 		Duration: 3.4 	Total: 5090.01 (sec.)


In [126]:
MY_MODEL = 'DietLP'
MY_TYPE='do-opl_12.9'

log('Creating model ' + MY_MODEL)

payload = {
    'name': MY_MODEL,
    'description': MY_MODEL,
    'type': MY_TYPE,
    'runtime': {
        'href': runtime
    }
}

r = requests.post(wml_credentials["url"] + '/v4/models', headers=mywmlheaders, json=payload)
res = json.loads(r.text)
model_uid = res['metadata']['guid']
model_rev = res['metadata']['href'].split('=')[-1]

print ("model_uid: " + model_uid)

log('Created model')

Creating model DietLP 		Duration: 1.04 	Total: 5091.05 (sec.)
model_uid: 580c45ec-2da6-426e-8403-7c8b42a907dc
Created model 		Duration: 0.52 	Total: 5091.56 (sec.)


In [127]:
payload = {
    'asset': {
        'href': '/v4/models/'+model_uid+'?rev='+model_rev
    },
    'name': MY_MODEL,
    'batch': {},
    'compute': {
        'name': 'S',
        'nodes': 1
    }
}
mywmlheaders['Content-Type'] = 'application/json'
r = requests.post(wml_credentials["url"] + '/v4/deployments', headers=mywmlheaders, json=payload)

res = json.loads(r.text)
deployment_uid = res['metadata']['guid']

print ("deployment_uid :" + deployment_uid)

log('Deployed model ' + MY_MODEL)

deployment_uid :29b51be7-0de4-42d4-8f1f-46da50a47c6e
Deployed model DietLP 		Duration: 1.04 	Total: 5092.6 (sec.)


In [136]:
log('Creating Job')

payload = {
    'deployment': {
            'href':'/v4/deployments/'+deployment_uid
    },
    'decision_optimization' : {
        'solve_parameters' : {
            'oaas.logAttachmentName':'log.txt',
            'oaas.logTailEnabled':'true',
           # 'oaas.resultsFormat': 'JSON'
        },
        'input_data_references': [
             {
                    'id':'model.mod',
                    'type': 's3',
                    'connection': {
                        'endpoint_url': cos_credentials["url"],
                        'access_key_id': cos_credentials["id"],
                        'secret_access_key': cos_credentials["key"]
                    },
                    'location': {
                        'bucket': cos_credentials["bucket"],
                        'path': 'model.mod'
                    }
                },
             {
                    'id':'Products.csv',
                    'type': 's3',
                    'connection': {
                        'endpoint_url': cos_credentials["url"],
                        'access_key_id': cos_credentials["id"],
                        'secret_access_key': cos_credentials["key"]
                    },
                    'location': {
                        'bucket': cos_credentials["bucket"],
                        'path': 'Products.csv'
                    }
                },
             {
                    'id':'Resources.csv',
                    'type': 's3',
                    'connection': {
                        'endpoint_url': cos_credentials["url"],
                        'access_key_id': cos_credentials["id"],
                        'secret_access_key': cos_credentials["key"]
                    },
                    'location': {
                        'bucket': cos_credentials["bucket"],
                        'path': 'Resources.csv'
                    }
                },
             {
                    'id':'Consumptions.csv',
                    'type': 's3',
                    'connection': {
                        'endpoint_url': cos_credentials["url"],
                        'access_key_id': cos_credentials["id"],
                        'secret_access_key': cos_credentials["key"]
                    },
                    'location': {
                        'bucket': cos_credentials["bucket"],
                        'path': 'Consumptions.csv'
                    }
                },
        ],
        'output_data_references': [
                {
                    'id':'plan.csv',
                    'type': 's3',
                    'connection': {
                        'endpoint_url': cos_credentials["url"],
                        'access_key_id': cos_credentials["id"],
                        'secret_access_key': cos_credentials["key"]
                    },
                    'location': {
                        'bucket': cos_credentials["bucket"],
                        'path': 'plan.csv'
                    }
                },
                {
                    'id':'log.txt',
                    'type': 's3',
                    'connection': {
                        'endpoint_url': cos_credentials["url"],
                        'access_key_id': cos_credentials["id"],
                        'secret_access_key': cos_credentials["key"]
                    },
                    'location': {
                        'bucket': cos_credentials["bucket"],
                        'path': 'log.txt'
                    }
                }
            ]
    }
}
r = requests.post(wml_credentials["url"] + '/v4/jobs', headers=mywmlheaders, json=payload)
res = json.loads(r.text)
job_uid = res['metadata']['guid']

print ("JobId: " + job_uid)

Creating Job 		Duration: 12.15 	Total: 5397.09 (sec.)
JobId: 4bdec2dc-00de-4c67-b7bf-a1256d705c2d


In [137]:
from time import sleep

# MONITOR

execution_status = ""

while ((execution_status != "failed") & (execution_status != "completed")):
    r = requests.get(wml_credentials["url"] + '/v4/jobs/' + job_uid, headers=mywmlheaders)
    res = json.loads(r.text)
    sleep(1)
    execution_status = res['entity']['decision_optimization']['status']['state']
    log('Executing ' + execution_status)

log('End of solve')

Executing running 		Duration: 1.67 	Total: 5398.76 (sec.)
Executing running 		Duration: 1.11 	Total: 5399.87 (sec.)
Executing completed 		Duration: 1.08 	Total: 5400.96 (sec.)
End of solve 		Duration: 0.0 	Total: 5400.96 (sec.)


In [118]:
client.deployments.delete(deployment_uid )

{"trace":"253fa79f765f69f5c6db3e113b2fd55a","errors":[{"code":"deployment_does_not_exists","message":"Deployment for model '6bd2d5a7-2044-4b4c-a220-415e43da3cc2' doesn't exists. Cannot delete non existing deployment"}]}


'FAILED'